In [1]:
from langchain_together import ChatTogether
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.memory import ConversationBufferMemory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.schema import ChatMessage, HumanMessage, AIMessage
from langchain_community.document_loaders import UnstructuredURLLoader
import requests
from bs4 import BeautifulSoup
from langchain.schema import Document
import PyPDF2
import gradio as gr
import uuid
from gtts import gTTS
import io
import speech_recognition as sr
import os

/home/mohammedorabi2002/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import requests
from bs4 import BeautifulSoup
from langchain.schema import Document

# List of links to scrape
links = [
    "https://www.ninds.nih.gov/health-information/stroke/stroke-overview",
    "https://www.ninds.nih.gov/health-information/stroke/prevention",
    "https://www.ninds.nih.gov/health-information/stroke/signs-and-symptoms",
    "https://www.ninds.nih.gov/health-information/stroke/assess-and-treat",
    "https://www.ninds.nih.gov/health-information/stroke/assess-and-treat/nih-stroke-scale",
    "https://www.ninds.nih.gov/health-information/stroke/recovery",
    "https://www.ninds.nih.gov/health-information/stroke/research",
]

# Function to scrape content using get_text()
def scrape_content(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        content = soup.get_text(separator=" ", strip=True)
        return {"content": content, "url": url}
    else:
        print(f"Failed to scrape {url}, status code: {response.status_code}")
        return None

In [6]:
documents = []
for link in links:
    data = scrape_content(link)
    if data:
        doc = Document(
            page_content=data["content"],
            metadata={"url": data["url"]})
        documents.append(doc)

In [2]:
embedding_model = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-base-en-v1.5"
)

In [ ]:
# vectorstore = Chroma.from_documents(
#     documents=documents,
#     embedding=embedding_model,
#     persist_directory="Stroke_vdb"
# )

In [3]:
vectorstore = Chroma(
    embedding_function=embedding_model,
    persist_directory="Stroke_vdb"
)

/tmp/ipykernel_1030/578217047.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [4]:
llm = ChatTogether(
    model = "meta-llama/Llama-3-70b-chat-hf",
    temperature=0.0,
    max_tokens=None,
    streaming=True,
    api_key="1b4b0c2624f2a3f595a50d4da9424898a53a23f824fda9b1651dfa895edfa2ac"
)

In [5]:
contextualize_q_system_prompt = (
    """Given a chat history and the latest user question
    which might reference context in the chat history,
    formulate a response which can be understood and clear
    without the chat history. Do NOT answer the question,
    """
)

retrieval = vectorstore.as_retriever(search_kwargs={'k': 3})

context_prompt = ChatPromptTemplate(
    [
        ('system',contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ('human','{input}')
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm,
    retrieval,
    context_prompt,
)

In [6]:
with open("prompt.txt",'r')as f:
    content = f.read()

    context = "{context}"

    prompt = f'("""\n{content.strip()}\n"""\n"{context}")'


In [7]:
qa_prompt = ChatPromptTemplate(
    [
        ('system',prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}")
    ]
)

In [8]:
ques_answer_chain = create_stuff_documents_chain(
    llm,
    qa_prompt
)

rag_chain = create_retrieval_chain(
    history_aware_retriever,
    ques_answer_chain
)

In [9]:
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [10]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
    max_tokens_limit=500,
    temperature=0.0
)

In [11]:
def chatbot_response(message, history):
    """Generate chatbot response with streaming"""
    session_id = str(uuid.uuid4())
    
    try:
        # Create a list to store streaming chunks
        full_response = ""
        
        # Stream the response
        for chunk in conversational_rag_chain.stream(
            {"input": message},
            config={"configurable": {"session_id": session_id}}
        ):
            if answer_chunk := chunk.get("answer"):
                full_response += answer_chunk
        
        # Update history with the new message and full response
        updated_history = history + [[message, full_response]]
        
        return updated_history, ""
    
    except Exception as e:
        error_response = f"I apologize, but there was an error processing your message: {str(e)}"
        updated_history = history + [[message, error_response]]
        return updated_history, ""

In [12]:
def extract_text(pdf_file: str):
    reader = PyPDF2.PdfReader(pdf_file)
    text = ""
    for pag_num in range(len(reader.pages)):
        page = reader.pages[pag_num]
        text += page.extract_text()
        return text

In [13]:
def text_to_speech(text):
    """Convert text to speech"""
        # Create a temporary MP3 file
    tts = gTTS(text=text, lang='en')
    mp3_fp = io.BytesIO()
    tts.write_to_fp(mp3_fp)
    mp3_fp.seek(0)
    return mp3_fp

In [14]:
def speech_to_text():
    """Convert speech to text using microphone"""
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening... Speak now.")
        recognizer.adjust_for_ambient_noise(source, duration=1)
        audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio)
            return text
        except sr.UnknownValueError:
            return "Sorry, could not understand audio"
        except sr.RequestError as e:
            return f"Could not request results; {e}"

In [15]:
def analyze_medical_report(text):
    """
    Analyze medical report text with a structured, easy-to-read output
    
    Prompt Design:
    - Clear instructions for interpretation
    - Structured output format
    - Focus on readability and key insights
    """
    detailed_prompt = f"""
    You are a medical report analysis assistant. Your task is to:
    
    1. Carefully review the medical report text
    2. Identify key medical values and their significance
    3. Provide a clear, concise interpretation in a user-friendly format
    4. Follow this output structure strictly:
    
    Medical Value Interpretation Guide:
    - If a value is outside normal range, indicate:
      * The specific value
      * Whether it's high or low
      * Potential implications (in simple language)
    - Use clear, non-technical language
    - Avoid medical jargon
    - Provide actionable insights
    
    Example Output Format:
    ```
    Glucose Level: 228.69 mg/dL
    ⚠️ Status: High
    Interpretation: Your glucose level is elevated, which may indicate:
    - Potential pre-diabetic condition
    - Need for dietary adjustments
    - Recommend consulting your healthcare provider

    Recommendation: Schedule a follow-up blood test
    ```

    Report Text:
    {text}

    Your Analysis:
    """
    
    try:
        response = ""
        for chunk in conversational_rag_chain.stream(
            {"input": detailed_prompt},
            config={"configurable": {"session_id": "medical_report_analysis"}}
        ):
            if answer_chunk := chunk.get("answer"):
                response += answer_chunk
                yield answer_chunk  
        
        return response
    except Exception as e:
        yield f"Error analyzing report: {str(e)}"


In [16]:
def create_gradio_interface():
    with gr.Blocks(theme='soft') as demo:
        gr.Markdown("# 🩺 Medical AI Assistant")
        
        with gr.Tab("Medical Report Analysis"):
            with gr.Row():
                with gr.Column():
                    pdf_input = gr.File(label="Upload Medical PDF", type="filepath")
                    analyze_btn = gr.Button("Analyze Report")
                
                with gr.Column():
                    report_output = gr.Textbox(label="Report Analysis", interactive=False)
                    audio_output = gr.Audio(label="Audio Analysis", type="filepath")
            
            # Report Analysis Processing
            analyze_btn.click(
                fn=lambda pdf: (
                    list(analyze_medical_report(extract_text(pdf)))[0], 
                    text_to_speech(list(analyze_medical_report(extract_text(pdf)))[0])
                ),
                inputs=pdf_input, 
                outputs=[report_output, audio_output]
            )
        
        with gr.Tab("Medical Chat Assistant"):
            with gr.Row():
                with gr.Column(scale=3):
                    # Text Input
                    msg = gr.Textbox(label="Your Message")
                
                with gr.Column(scale=1):
                    # Voice Input Button
                    voice_btn = gr.Button("🎤 Voice Input")
                
            # Chatbot with Streaming Response
            chatbot = gr.Chatbot(
                label="Medical Chat",
                bubble_full_width=False,
                layout='bubble'
            )
            submit_btn = gr.Button("Send")
            clear_btn = gr.Button("Clear Chat")
            
            # Voice Input Handler
            voice_btn.click(
                fn=speech_to_text, 
                outputs=msg
            )
            
            # Chatbot Interaction with Streaming
            submit_btn.click(
                chatbot_response, 
                [msg, chatbot], 
                [chatbot, msg]
            )
            
            # Clear Chat Functionality
            clear_btn.click(
                lambda: None, 
                None, 
                chatbot
            )
        
        with gr.Tab("Speech Interaction"):
            speech_input = gr.Textbox(label="Additional Speech Input Options")
            mic_btn = gr.Button("Start Advanced Speech Input")
            
            # Advanced Speech-to-Text
            mic_btn.click(
                fn=speech_to_text, 
                outputs=speech_input
            )
    
    return demo

# Launch the Gradio App
if __name__ == "__main__":
    app = create_gradio_interface()
    app.launch(share=True)

/home/mohammedorabi2002/miniconda3/lib/python3.12/site-packages/gradio/components/chatbot.py:237: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://c29af21263974291ba.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/home/mohammedorabi2002/miniconda3/lib/python3.12/site-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mohammedorabi2002/miniconda3/lib/python3.12/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mohammedorabi2002/miniconda3/lib/python3.12/site-packages/gradio/blocks.py", line 2043, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mohammedorabi2002/miniconda3/lib/python3.12/site-packages/gradio/blocks.py", line 1590, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mohammedorabi2002/miniconda3/lib/python3.12/site-packages/an

Listening... Speak now.


ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM sysdefault
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No

Listening... Speak now.


ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM sysdefault
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No

Listening... Speak now.
